# Time Series Analysis: "The Final Project"

`End? No, the journey doesn't end here. Death is just another path. One that we all must take.
-J.R.R. Tolkien, The Return of the King`

---

## Libraries

In [6]:
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
import statsmodels.api as sm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from pandas.plotting import register_matplotlib_converters
from IPython.display import display
from tsa_tools import *  # Hehe

register_matplotlib_converters()
sns.set_style('darkgrid')

np.set_printoptions(precision=4)
pd.set_option('precision', 4)

---

## M5 Forecasting

For this "final project", we will be forecasting the <b><u>level 9</b></u> series (unit sales of all products, aggregated for each store and department).

Load `sales_train_evaluation.csv` and use observations from `d_1 to d_1913` for training and `d_1914 to d_1941` for testing.

In [3]:
df_calendar = pd.read_csv('../data/m5/calendar.csv')
df_sales = pd.read_csv('../data/m5/sales_train_evaluation.csv')
df_weights = pd.read_csv('../data/m5/weights_validation.csv')
# display(df_calendar, df_sales, df_weights)

In [4]:
train_df = (df_sales.set_index([*df_sales.columns[5::-1]]).T
           .set_index(pd.DatetimeIndex(df_calendar.date)[:1941]).iloc[:-28])
test_df = (df_sales.set_index([*df_sales.columns[5::-1]]).T
           .set_index(pd.DatetimeIndex(df_calendar.date)[:1941]).iloc[-28:])
# display(train_df, test_df)

In [5]:
levels = {
    1: None,
    2: "state_id",
    3: "store_id",
    4: "cat_id",
    5: "dept_id",
    6: ["state_id", "cat_id"],
    7: ["state_id", "dept_id"],
    8: ["store_id", "cat_id"],
    9: ["store_id", "dept_id"],
    10: "item_id",
    11: ["state_id", "item_id"],
    12: ["store_id", "item_id"]
}

In [6]:
train_df_9 = timeSeriesFiltering(
    train_df.sum(axis='columns', level=levels[9]), lower=10)
trainOG_df_9 = train_df.sum(axis='columns', level=levels[9])
test_df_9 = test_df.sum(axis='columns', level=levels[9])
weights_df_9 = (df_weights
                .loc[df_weights['Level_id'] == 'Level9']
                .set_index(['Agg_Level_1', 'Agg_Level_2'])[['Weight']])

---

## Part 5. King of the Hill (20 pts.)

Using whatever methods/models you desire, beat the best `WRMSSE` score in Q5.

<b><u>Do not tune your model using the test set!</b></u> If you do, you will not get points for this part.

### Q8. (10 pts.)

Describe your methodology here. 

Points will be awarded for <b>aesthetics</b> (ex. use of diagrams), <b>ease of reading</b>, <b>clarity</b>, and <b>brevity</b>.

Points will be deducted for <b>excessively long</b> walls of text and descriptions.

<img src="Slide1.png" width="800" />
<img src="Slide2.png" width="800" />
<img src="Slide3.png" width="800" />
<img src="Slide4.png" width="800" />
<img src="Slide5.png" width="800" />
<img src="Slide6.png" width="800" />

### Q9. (10 pts.)

This part is for your actual code.

In [7]:
tscv = TimeSeriesSplit(n_splits=3, test_size=28, )
methods = {
    "Seasonal Naive": {
        'meta': 'base', 
        'model':BaseFuncModel(snaivef, m=7)},
    "SES": {
        'meta': 'stat',
        'model': StatsModelsWrapper(ETSModel, trend=None, seasonal=None)},
    "Holt's Linear": {
        'meta': 'stat',
        'model': StatsModelsWrapper(ETSModel, trend='add', seasonal=None)},
    "Additive Holt-Winters": {
        'meta': 'stat',
        'model': StatsModelsWrapper(
            ETSModel, seasonal_periods=7, trend='add', seasonal='add')},
    "RecursiveRegressor(LGBMRegressor)": {
        'meta': 'ml_recursive',
        'model': RecursiveRegressor(
            lgb.LGBMRegressor(random_state=1, w=84, h=28, n_jobs=-1))},
    "MultiOutputRegressor(LGBMRegressor)":{
        'meta': 'ml_direct',
        'model': MultiOutputRegressor(
            lgb.LGBMRegressor(random_state=1, n_jobs=-1),n_jobs=-1)},
    "RegressorChain(LGBMRegressor)": {
        'meta': 'ml_direct',
        'model': RegressorChain(
            lgb.LGBMRegressor(random_state=1, n_jobs=-1))},
    "Combo(LGBMRegressor)": {
        'meta': 'combo',
        'model': [
            RecursiveRegressor(
                lgb.LGBMRegressor(random_state=1, w=84, h=28, n_jobs=-1)),
            MultiOutputRegressor(
                lgb.LGBMRegressor(random_state=1, n_jobs=-1),n_jobs=-1)
        ]}
}

In [ ]:
for col in train_df_9:
    print(col)
    methods = evaluate_methods(
        methods,
        X=train_df_9[col],
        XOG=trainOG_df_9[col],
        tscv=tscv,
        col=col,
        w=84,
        h=28)

In [10]:
rmse_scores = score_reveal(methods)
rmse_scores

Seasonal Naive       SES  Holt's Linear  \
CA_1 HOBBIES_1          112.5315  102.1342       102.2342   
     HOBBIES_2           17.1085   13.3313        13.4002   
     HOUSEHOLD_1        124.7181  182.9850       183.3727   
     HOUSEHOLD_2         42.9325   54.9331        54.7421   
     FOODS_1             96.5581  115.3984       113.5375   
...                          ...       ...            ...   
WI_3 HOUSEHOLD_1        119.6917  151.5976       151.4656   
     HOUSEHOLD_2         28.4814   38.4835        38.4900   
     FOODS_1             85.1284   95.9297        96.3109   
     FOODS_2            192.7013  167.3461       166.0414   
     FOODS_3            401.1326  471.3999       490.9567   

                  Additive Holt-Winters  RecursiveRegressor(LGBMRegressor)  \
CA_1 HOBBIES_1                  89.0478                            88.1818   
     HOBBIES_2                  11.8658                            13.7598   
     HOUSEHOLD_1                80.5718                            77.1174   
     HOUSEHOLD_2                31.1680                            32.8088   
     FOODS_1                    92.5809                            83.4848   
...                                 ...                                ...   
WI_3 HOUSEHOLD_1               100.3098                            96.1385   
     HOUSEHOLD_2                27.9321                            26.6128   
     FOODS_1                    86.2128                            67.6526   
     FOODS_2                   180.1888                           109.7245   
     FOODS_3                   307.5882                           272.5932   

                  MultiOutputRegressor(LGBMRegressor)  \
CA_1 HOBBIES_1                                91.9518   
     HOBBIES_2                                14.0885   
     HOUSEHOLD_1                              78.3647   
     HOUSEHOLD_2                              34.2279   
     FOODS_1                                  71.9233   
...                                               ...   
WI_3 HOUSEHOLD_1                              96.7119   
     HOUSEHOLD_2                              29.6885   
     FOODS_1                                  65.7408   
     FOODS_2                                 103.9821   
     FOODS_3                                 261.8917   

                  RegressorChain(LGBMRegressor)  Combo(LGBMRegressor)  
CA_1 HOBBIES_1                          90.5094               89.2338  
     HOBBIES_2                          12.5092               13.3713  
     HOUSEHOLD_1                        76.8850               76.4375  
     HOUSEHOLD_2                        34.3215               33.1396  
     FOODS_1                            74.6178               75.9609  
...                                         ...                   ...  
WI_3 HOUSEHOLD_1                        95.9121               93.5357  
     HOUSEHOLD_2                        27.0664               27.0069  
     FOODS_1                            64.0030               65.1575  
     FOODS_2                           111.8733              104.1259  
     FOODS_3                           259.7017              255.9858  

[70 rows x 8 columns]

In [11]:
best_models = rmse_scores.apply(lambda x: x.idxmin(), axis=1)
best_models

CA_1  HOBBIES_1        RecursiveRegressor(LGBMRegressor)
      HOBBIES_2                    Additive Holt-Winters
      HOUSEHOLD_1                   Combo(LGBMRegressor)
      HOUSEHOLD_2                  Additive Holt-Winters
      FOODS_1        MultiOutputRegressor(LGBMRegressor)
                                    ...                 
WI_3  HOUSEHOLD_1                   Combo(LGBMRegressor)
      HOUSEHOLD_2      RecursiveRegressor(LGBMRegressor)
      FOODS_1              RegressorChain(LGBMRegressor)
      FOODS_2        MultiOutputRegressor(LGBMRegressor)
      FOODS_3                       Combo(LGBMRegressor)
Length: 70, dtype: object

In [12]:
best_models.value_counts()

Additive Holt-Winters                  27
Combo(LGBMRegressor)                   10
RegressorChain(LGBMRegressor)          10
RecursiveRegressor(LGBMRegressor)      10
MultiOutputRegressor(LGBMRegressor)     9
SES                                     3
Holt's Linear                           1
dtype: int64

In [15]:
model = ensemble2(col_assignment=best_models.to_dict(), methods=methods, w=84, h=28)
model.fit(train_df_9)

In [ ]:
df_pred_9_ensemble2 = model.predict(trainOG_df_9)
df_pred_9_ensemble2

CA_1                                                                   \
   HOBBIES_1 HOBBIES_2 HOUSEHOLD_1 HOUSEHOLD_2   FOODS_1   FOODS_2    FOODS_3   
0   437.5033   39.0957    763.9959    207.2370  266.7877  509.9272  1925.1010   
1   392.2056   38.7922    654.6803    189.0628  242.2286  405.2741  1777.0478   
2   423.5902   40.4461    619.8833    189.6468  283.8521  348.0841  1754.3820   
3   406.6826   39.1083    593.2084    197.6096  300.6551  372.3611  1844.6085   
4   453.2403   43.9026    688.9625    226.5993  344.9472  518.5610  2073.3072   
5   580.0852   50.3332   1028.9997    290.3485  400.7362  592.5850  2682.3390   
6   516.6611   52.1988   1084.4108    280.3655  351.8076  662.2098  2926.0347   
7   461.4200   39.1899    774.8963    207.5895  266.4044  510.1214  2140.1317   
8   406.5608   38.8864    669.6527    189.4153  274.1547  474.5079  1948.6778   
9   439.8678   40.5403    642.2546    189.9993  250.8226  491.0791  1899.5062   
10  410.4082   39.2025    634.9184    197.9621  300.3507  478.0818  1845.9984   
11  504.4875   43.9968    804.9454    226.9518  357.3245  585.9167  2131.2749   
12  589.0777   50.4274   1058.8714    290.7011  457.7425  713.5284  2625.3539   
13  577.3892   52.2930    927.3517    280.7180  373.2274  710.9126  2985.1508   
14  464.7283   39.2841    753.1826    207.9420  285.9629  572.7136  2141.5748   
15  477.6738   38.9806    651.8784    189.7678  289.3044  485.3668  1870.4277   
16  449.3603   40.6345    607.0223    190.3518  340.1288  457.0679  1842.7475   
17  408.6097   39.2967    641.7250    198.3146  350.8922  402.1455  1847.9103   
18  431.7140   44.0910    717.3441    227.3043  370.4809  458.6446  2082.2765   
19  575.4122   50.5216   1050.0140    291.0536  460.1207  620.8623  2577.9228   
20  559.2288   52.3872   1041.3984    281.0705  424.8669  737.3921  2902.4068   
21  513.5346   39.3783    764.7825    208.2945  343.1479  520.1686  1932.6558   
22  412.3940   39.0748    649.4966    190.1204  326.5434  444.9639  1768.1819   
23  420.7540   40.7287    629.8517    190.7044  317.8152  427.8329  1733.0222   
24  461.0836   39.3909    637.7166    198.6671  315.9579  399.3173  1841.6854   
25  450.1613   44.1852    751.6506    227.6569  366.6447  495.0518  2140.8349   
26  577.6936   50.6158    997.5750    291.4061  468.6073  549.7108  2603.9882   
27  568.9221   52.4814   1059.0841    281.4230  380.0597  629.6233  2887.6528   

        CA_2                        ...      WI_2                        \
   HOBBIES_1 HOBBIES_2 HOUSEHOLD_1  ...   FOODS_1    FOODS_2    FOODS_3   
0   320.4134   39.3100    666.1571  ...  320.4731   964.3415  1868.7333   
1   349.0217   34.7580    602.7528  ...  333.5456   721.1611  1640.8760   
2   280.4951   41.5079    602.8867  ...  343.4532   710.5954  1680.4127   
3   350.8923   37.9551    599.6201  ...  339.3687   802.8031  1688.3539   
4   427.0443   41.1714    765.0730  ...  380.7957   713.4821  2152.6675   
5   467.5930   52.6185   1077.7876  ...  405.5013   835.2595  2311.1217   
6   456.7799   48.7608   1117.9953  ...  349.6992  1033.3559  2383.4232   
7   318.7420   32.7714    667.6338  ...  320.8609  1306.3273  2591.0659   
8   318.5709   37.6144    604.2295  ...  333.9334  1380.8530  2449.1456   
9   319.1154   39.6117    604.3635  ...  343.8410  1586.7742  2655.3447   
10  362.4959   41.2234    601.0968  ...  339.7565  1421.9335  2664.2943   
11  394.4333   51.5402    766.5497  ...  381.1835  1454.7442  2765.1528   
12  476.4257   50.1408   1079.2643  ...  405.8891  1655.6166  2993.4483   
13  457.2294   49.7115   1119.4720  ...  350.0871  1507.0139  2782.9057   
14  305.3235   33.8597    669.1105  ...  321.2488  1624.9390  2850.7330   
15  292.2392   35.7904    605.7062  ...  334.3212  1710.7301  2563.2396   
16  309.3083   38.7258    605.8402  ...  344.2288  1311.4817  2396.4343   
17  304.2859   35.1682    602.5735  ...  340.1444  1597.2741  2512.8257   
18  362.2918   48.0371    768.0264  ...  381.5714  1642.1615  2845.3863   
19

In [17]:
df_res_9_esemble2 = rateMyForecast(
    trainOG_df_9, test_df_9, df_pred_9_ensemble2)['RMSSE']
df_res_9_esemble2.index = pd.MultiIndex.from_tuples(
    df_res_9_esemble2.index, names=['Agg_Level_1', 'Agg_Level_2'])
df_res_9_esemble2

Agg_Level_1  Agg_Level_2
CA_1         HOBBIES_1      0.7412
             HOBBIES_2      0.6931
             HOUSEHOLD_1    0.3584
             HOUSEHOLD_2    0.5141
             FOODS_1        0.5867
                             ...  
WI_3         HOUSEHOLD_1    0.7166
             HOUSEHOLD_2    0.7675
             FOODS_1        1.6214
             FOODS_2        0.8506
             FOODS_3        0.4849
Name: RMSSE, Length: 70, dtype: float64

### THE RESULT

In [18]:
df_res_9_esemble2.multiply(weights_df_9.squeeze(), axis=0).sum()

0.7259427322254518